In [ ]:
!pip install xlrd

In [1]:
import pandas as pd

df = pd.read_excel('GSAF5.xls')
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,14th October,2025.0,Unprovoked,Columbia,"Bolivar, del Isolate",Catagena Province,Swimming with sharks,Male child,M,14,...,Nurse shark,Kevin McMurray Trackingsharks.com Andy Currie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11th October,2025.0,Unprovoked,Australia,Queensland,Cook Esplanade Thursday Island,Fishing/swimming,Samuel Nai,M,14,...,Tiger or Bull shark,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7th October,2025.0,Unprovoked,Australia,South Australia,Kangaroo Island,Surfing,Lee Berryman,M,50+,...,Bronze whaler?,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29th September,2025.0,Unprovoked,USA,Off California,Catalina Island,Swimming,Christopher Murray,M,54,...,unknown 1.2m shark,Todd Smith: Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27th September,2025.0,Provoked,Costa Rica,NaN,Cocos Islands,Diving-Tagging sharks,Dr. Mauricio Hoyos,M,48,...,Tiger shark 4m,Todd Smith: Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.tail(10)

In [ ]:
df.sample(10)

In [ ]:
df.columns.to_list()

In [ ]:
df.columns

In [ ]:
df.info()

In [7]:
df['Fatal Y/N'].nunique()

3

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

RODRIGO CODE

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

In [8]:
df['Fatal Y/N'].value_counts()

Fatal Y/N
N          4924
Y          1483
Unknown     643
Name: count, dtype: int64

Cleaning 'Fatal' Data Column:

In [2]:
df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()

df['Fatal Y/N'] = df['Fatal Y/N'].replace({
    'N ': 'N',
    ' N': 'N',
    'UNKNOWN': 'Unknown',  
    'F': 'Unknown',
    'Y ': 'Y',
    'M': 'Unknown', 
    '2017': 'Unknown',
    'Y x 2': 'Unknown',
    '0': 'Unknown'
})

df.loc[~df['Fatal Y/N'].isin(['Y', 'N']), 'Fatal Y/N'] = 'Unknown'



In [3]:
df['Fatal Y/N'].value_counts()

Fatal Y/N
N          4924
Y          1483
Unknown     643
Name: count, dtype: int64

Adding clean 'Injury_Class' column to the data set:

In [4]:
df['Injury'].nunique()

4167

In [5]:
df['Injury'] = df['Injury'].fillna('Unknown').astype(str).str.lower().str.strip()

def classify_injury(text):
    text = str(text).lower().strip()
    if any(word in text for word in ['no injury', 'uninjured', 'none', 'NaN']):
        return 'No Injury'
    elif 'survived' in text or 'escaped' in text or 'unharmed' in text:
        return 'Survived'
    elif any(word in text for word in [
        'body not recovered', 'fatal', 'died', 'death', 'deceased',
        'human remains', 'drown', 'lost at sea', 'presumed dead', 'remains of the men'
    ]):
        return 'Fatal Wounds'
    elif any(word in text for word in [
        'minor', 'scratch', 'abrasion', 'bruise', 'small', 'superficial',
        'cut', 'scrape', 'nipped', 'grazed', 'bitten but not serious',
        'bumped', 'knocked', 'struck', 'collision', 'impact', 'puncture', 'non-life-threatening' 
    ]):
        return 'Minor Wounds'
    elif any(word in text for word in [
        'amputation', 'severe', 'major', 'deep', 'critical', 'multiple',
        'serious', 'massive', 'wound', 'bite', 'bitten', 'injury to',
        'injuries to', 'injured', 'injuries', 'damage', 'torn',
        'arm severed', 'leg severed', 'lacerat', 'tissue loss'
    ]):
        return 'Severe Wounds'
    else:
        return 'Unknown'

df['Injury_Class'] = df['Injury'].apply(classify_injury)
df['Injury_Class'] = df['Injury_Class'].str.title()

In [6]:
df['Injury_Class'].value_counts()

Injury_Class
Severe Wounds    3110
Fatal Wounds     1615
No Injury         884
Minor Wounds      816
Unknown           457
Survived          168
Name: count, dtype: int64

In [74]:
df[df['Injury_Class'] == 'Unknown']['Injury'].sample(30, random_state=42)

5143    part of hand removed by shark he had caught pr...
846                                 fingernail pulled off
5521                                           no details
4233    bones and brightly colored bangles recovered  ...
3577                               contusion of left foot
4058                    sharks reportedly bit legs & fins
3812                            toothmarks on head & neck
2914    he grabbed shark's tail, shark bit his chest &...
6283       disappeared, but shark involvement unconfirmed
534       partial remains recovered 12/26/2019 from shark
2085                                              unknown
3742    disappeared while fishing from shore. leg foun...
3384                                           no details
5712                                              unknown
5916        arms broken by hooked shark provoked incident
1393                   two fingers lost provoked incident
5805               body of woman recovered from 15' shark
4902          

In [76]:
df['Injury_Class'].value_counts()

Injury_Class
Severe Wounds    3110
Fatal Wounds     1615
No Injury         884
Minor Wounds      816
Unknown           457
Survived          168
Name: count, dtype: int64

In [71]:
df['Injury_Class'].sample(50, random_state=42).tolist()

['Severe Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Fatal Wounds',
 'Minor Wounds',
 'Severe Wounds',
 'Fatal Wounds',
 'Fatal Wounds',
 'Severe Wounds',
 'Fatal Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Minor Wounds',
 'Fatal Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Minor Wounds',
 'No Injury',
 'Severe Wounds',
 'Fatal Wounds',
 'Fatal Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Minor Wounds',
 'Severe Wounds',
 'No Injury',
 'Minor Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Fatal Wounds',
 'Severe Wounds',
 'Fatal Wounds',
 'Fatal Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'No Injury',
 'Fatal Wounds',
 'Minor Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Fatal Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Severe Wounds',
 'Severe Wounds']

In [ ]:
df['Injury'].value_counts()

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

In [ ]:
# Divya's work
# Cleaning up 'activity' column in the dataset

df['Activity'].nunique() #unique values in the activity column

In [ ]:
df['Activity'].unique()


In [ ]:
df['activity_clean'] = (
    df['Activity']
    .str.lower() #lowercase
    .str.strip() #remove spaces
    .str.replace(r'[^a-z\s]', '', regex=True)  #remove punctuation/numbers
)


In [ ]:
import numpy as np

def categorize_activity(text):
    if pd.isna(text): #handling 'NaN' values and labelling them as unknown
        return "unknown"
    if any(word in text for word in ["surf", "bodyboard", "paddle", "boogie", "body boarding", "kiteboarding", "foilboarding", "skimboarding", "wakeboarding"]): #if the word is present in the text, then loop over each word and replace with return 'string'
        return "Surfing"
    elif any(word in text for word in ["swim", "bathing", "snorkel", "rescue", "float", "splash", "swimming"]):
        return "Swimming"
    elif any(word in text for word in ["fish", "spearfish", "net", "catch", "line", "fishing", "spear", "scalloping", "lobstering", "hunt", "clamming"]):
        return "Fishing"
    elif any(word in text for word in ["dive", "scuba", "freediv", "underwater", "research", "investigat", "pearl", "recover", "diving"]): #'investigat' covers terms like investigating / investigation / investigator..
        return "Diving"
    elif any(word in text for word in ["boat", "kayak", "sail", "ship", "vessel", "frigate", "dinghy", "canoe", "race", "compet", "rowing", "watercraft", "jet ski", "paddling", "sculling", "raft", "yacht"]):
        return "Boating"
    elif any(word in text for word in ["walk", "stand", "wade", "reef", "shore", "beach", "adrift", "wading", "tread"]):
        return "Wading"
    elif any(word in text for word in ["sea disaster", "aircraft", "boeing", "wreck", "hurricane", "tsunami", "earthquake", "disaster", "plunged", "sank", "destroyed", "overboard", "suicide", "air", "petting", "capsize", "swept", "help", "ride", "sunk", "went down", "crash", "sinking"]):
        return "Catastrophe"
    else:
        return "Other Activity"

df['activity_group'] = df['activity_clean'].apply(categorize_activity) #new column 'activity group' for categorised activity


In [ ]:
print("Original:", df['Activity'].nunique()) #original dataset
print("Cleaned:", df['activity_clean'].nunique()) #activity column after removing spaces, fixing inconsistent letter cases and removing punctutations
print("Grouped:", df['activity_group'].nunique()) #after categorising all the activity under specific keywords

In [ ]:
df[df['activity_group'] == 'Other Activity']['activity_clean'].sample(40, random_state=42).tolist()
#sample of 'other' category to improve mapping and grouping


In [ ]:
df['activity_group'].value_counts()

In [22]:
#Cleaning and formatting the 'date' column and extracting 'month' from it
df["Date"].head(30)


0            14th October
1            11th October
2             7th October
3          29th September
4          27th September
5           6th September
6           1st September
7             30th August
8             18th August
9             17th August
10            16th August
11             7th August
12             1st August
13              28th July
14              25th July
15              22nd July
16              20th July
17              19th July
18              18th July
19              15th July
20              6th July 
21               6th July
22               4th July
23              29th June
24              25th June
25              22nd June
26              17th June
27    2025-06-11 00:00:00
28               31st May
29    2025-05-29 00:00:00
Name: Date, dtype: object

In [23]:
df["Date"].tail(30)


7020                                         World War II
7021                                          Before 1905
7022                              A few years before 1938
7023                                              No date
7024                                          Early 1930s
7025                                          Before 1927
7026                                  Between 1918 & 1939
7027                                              No date
7028                                              No date
7029                                              No date
7030                                           1920 -1923
7031                                          Before 1921
7032                                          Before 1911
7033                                          Before 1921
7034                                          Before 1921
7035                                          Before 1917
7036                                   Before 17-Jul-1916
7037    No dat

In [19]:
df['Date'] = df['Date'].astype(str)


In [20]:
import numpy as np

def extract_month_or_nan(value):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    value_str = str(value)
    for month in months:
        if month in value_str:
            return month
    return np.nan  #assign NaN if no month found

df['Extracted_Month_NaN'] = df['Date'].apply(extract_month_or_nan)

count_months = df['Extracted_Month_NaN'].count() #count extracted months (non-null values)
print(f'Number of extracted months: {count_months}')

count_nan = df['Extracted_Month_NaN'].isna().sum() #count how many NaN values were assigned
print(f'Number of non-month values: {count_nan}')


Number of extracted months: 6449
Number of non-month values: 601


In [27]:
df['Extracted_Month_NaN'].value_counts()

Extracted_Month_NaN
Jul    784
Aug    673
Sep    611
Jan    566
Jun    549
Oct    503
Apr    493
Dec    491
Mar    458
May    452
Nov    451
Feb    418
Name: count, dtype: int64